# [모듈 3.1] 전처리 스텝 개발 (SageMaker Model Building Pipeline 전처리 스텝)

이 노트북은 아래와 같은 목차로 진행 됩니다. 전체를 모두 실행시에 완료 시간은 약 5분-10분 소요 됩니다.

- 0. 전처리 개요 (SageMaker Processing 이용) 
- 1. 기본 세이지 메이커 정보 및 변수 로딩
- 2. 전처리 코드 확인
- 3. 전치리 스텝 개발 및 실행
    - 아래의 3단계를 진행하여 SageMaker Model Building Pipeline 에서 전치러 스텝 개발 함. 아래의 (1), (2) 단계는 옵션이지만, 실제 현업 개발시에 필요한 단계이기에 실행을 권장 드립니다.
        - (1) [옵션] 로컬 노트북 인스턴스에서 **[다커 컨테이너 없이]** 전처리 코드 실행
        - (2) [옵션] 로컬 노트북 인스턴스에서 **[다커 컨테이너를 통해서]** 전처리 코드 실행          
        - (3) [필수] SageMaker Model Building Pipeline 에서 전치러 스텝 개발 및 실행
    
---
### 노트북 커널
- 이 워크샵은 노트북 커널이 `conda_python3` 를 사용합니다. 다른 커널일 경우 변경 해주세요.
---

# 0. 전처리 개요 (SageMaker Processing 이용)

이 노트북은 세이지 메이커의 Processing Job을 통해서 데이터 전처리를 합니다. <br>
상세한 사항은 개발자 가이드를 참조 하세요. -->  [SageMaker Processing](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/processing-job.html)


### SageMaker Processing 개요
![sagemaker-processing-archi.png](img/sagemaker-processing-archi.png)

### 전처리 실행 요약
![Processing-1.png](img/Processing-1.png)

- 일반적으로 크게 아래 4가지의 스텝으로 진행이 됩니다.

    - (1) S3에 입력 파일 준비
    - (2) 전처리를 수행하는 코드 준비
    - (3) Projcessing Job을 생성시에 아래와 같은 항목을 제공합니다.
        - Projcessing Job을 실행할 EC2(예: ml.m4.2xlarge) 기술
        - EC2에서 로딩할 다커 이미지의 이름 기술
        - S3 입력 파일 경로
        - 전처리 코드 경로
        - S3 출력 파일 경로
    - (4) EC2에서 전치리 실행 하여 S3 출력 위치에 저장



# 1. 기본 세이지 메이커 정보 및 기본 변수 로딩
- 기본으로 사용하는 세이지 메이커 변수의 로딩 및 `%store -r` 을 통해서 이전 노트북에서 저장한 변수를 로딩 합니다.

In [1]:
#TODO: Remove store

In [2]:
import boto3
import sagemaker
import pandas as pd
from IPython.display import display as dp

region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()

# 2. 전처리 코드 확인

전처리 코드는 크게 아래와 같이 구성 되어 있습니다.
- 커맨드 인자로 전달된 변수 내용 확인
- 두개의 입력 파일(claims.csv, customers.csv)을 로딩하고, policy_id 로 조인하여 하나의 데이터 세트로 만듦
- 카테고리 피쳐를 원핫인코딩을 함.
- 숫자형 변수의 결측값에 대해서 채우는 전처리 수행
    - [알림] 이 워크샵은 XGBoost를 통해서 훈련을 합니다. XGBoost 같은 Tree 계열의 알고리즘은 피쳐의 스케일링(Scaling)이 꼭 필요하지 않습니다.
- 전치리 결과를 모두 결합 합니다.
- 훈련, 테스트 데이터 세트로 split_rate (예: 8:2) 로 분리 합니다.
    - [알림] 일반적으로 Fruad Detection의 훈련, 테스트 데이터의 분리는 시간의 순서에 따라 분리 합니다. 예를 들어서 2021.01.01 ~ 2021.12.31 의 데이터가 존재한다고 하면, 2021.01.01 ~ 2021.11.30 을 훈련 데이터, 2021.12.01 ~ 2021.12.31을 테스트 테이터로 분리 합니다. 이 워크샵에서 사용하고 있는 데이터 세트는 명시적인 날짜 정보가 없어서, 5000개의 의 policy_id에서 1~4,000 은 훈련 데이터, 4000 ~ 5,000은 테스트 데이터로 분리 하였습니다.

---

In [3]:
preprocessing_code = 'src/preprocessing.py'
%store preprocessing_code

Stored 'preprocessing_code' (str)


In [4]:
!pygmentize {preprocessing_code}

import argparse
import os
import requests
import tempfile
import subprocess, sys

import pandas as pd
import numpy as np
from glob import glob

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

import logging
import logging.handlers

def _get_logger():
    '''
    로깅을 위해 파이썬 로거를 사용
    # https://stackoverflow.com/questions/17745914/python-logging-module-is-printing-lines-multiple-times
    '''
    loglevel = logging.DEBUG
    l = logging.getLogger(__name__)
    if not l.hasHandlers():
        l.setLevel(loglevel)
        logging.getLogger().addHandler(logging.StreamHandler(sys.stdout))        
        l.handler_set = True
    return l  

logger = _get_logger()


def split_train_test(df, test_ratio=0.1):
    '''
    두 개의 데이터 세트로 분리
    '''
    total_rows = df.shape[0]
    train_end = int(total_rows * (1 - test_ratio))
    
    train_df = df[0:train_

# 3. 전치리 스텝 개발 및 실행
- 전처리 스텝의 개발이 현업에서 아래의 (1), (2), (3) 의 순서로 개발이 될 수 있습니다. 그래서 각 단계별로 실행을 진행 합니다.

## 3.1 개요
### (1) 로컬 노트북 인스턴스에서 **[다커 컨테이너 없이]** 전처리 코드 실행

- 이 단계의 목적은 현업 프로젝트의 개발시에 로직 확인, 디버깅이 수월하기 위해서 이 단계를 진행 합니다. 
- SageMaker Processing은 다커 컨테이너에서 실행이 되기에, 로컬에서 다커 환경과 비슷한 환경을 구성(예: 입력, 출력 위치)하여 실행합니다.


### (2) 로컬 노트북 인스턴스에서 **[다커 컨테이너를 통해서]** 전처리 코드 실행      
- 이 단계의 목적은 SageMaker Model Building Pipeline이 다커 컨테이너 형태로 실행이 되기에, 이 단계에서 다커 컨테이너에서 정상적으로 동작하는지를 확인하기 위해서 이 단계를 수행 합니다.
- 위의 (1) 단계에서 전처리 코드의 로직 확인이 되었기에, 실제 로컬 노트북 인스턴스에서 다커 컨테이너를 통해 전치리 코드를 수행 합니다.
- [알림] 로컬 모드 및 다커 참고 자료
    - 로컬모드 설명하는 블로그 자료 --> [Use the Amazon SageMaker local mode to train on your notebook instance](https://aws.amazon.com/blogs/machine-learning/use-the-amazon-sagemaker-local-mode-to-train-on-your-notebook-instance/)
    - TF, Pytorch, SKLean, SKLearn Processing JOb에 대한 로컬 모드 샘플 --> [Amazon SageMaker Local Mode Examples](https://github.com/aws-samples/amazon-sagemaker-local-mode)
    - Python SDK -->  [로컬모드 Python SDK](https://sagemaker.readthedocs.io/en/stable/overview.html#local-mode)
    - [SageMaker 에서 도커 컨테이너 사용하기](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/docker-containers.html)
    - [Hello 다커](https://github.com/mullue/hello-docker/blob/master/hello_docker.ipynb)
    
### (3) SageMaker Model Building Pipeline 에서 전치러 스텝 개발 및 실행
- 상세 사항은 여기에서 확인 하세요. --> [Amazon SageMaker 모델 구축 파이프라인](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/pipelines.html)
    
---    
## 3.2 실행    
### (1) 로컬 노트북 인스턴스에서 **[다커 컨테이너 없이]** 전처리 코드 실행

#### 로컬 환경 셋업 

- 로컬에서 테스트 하기 위해 세이지메이커의 다커 컨테이너와 같은 환경을 생성합니다.
- split_rate = 0.2 로 해서 훈련 및 테스트 데이터 세트의 비율을 8:2로 정합니다.

In [5]:
import os

# 도커 컨테이너의 출력 폴더와 비슷한 환경 기술
# 아래 경로 : opt/ml/processing/output
# 도커 경로 : /opt/ml/processing/output
base_output_dir = 'opt/ml/processing/output' 

# 도커 컨테이너의 입력 폴더와 비슷한 환경 기술
base_preproc_input_dir = 'opt/ml/processing/input'
os.makedirs(base_preproc_input_dir, exist_ok=True)

# 출력 훈련 폴더를 기술 합니다.
base_preproc_output_train_dir = 'opt/ml/processing/output/train/'
os.makedirs(base_preproc_output_train_dir, exist_ok=True)

# 출력 테스트 폴더를 기술 합니다.
base_preproc_output_test_dir = 'opt/ml/processing/output/test/'
os.makedirs(base_preproc_output_test_dir, exist_ok=True)


split_rate = 0.2

#### claims.csv, customers.csv 를 다커환경에서 사용하는 입력 경로로 복사

In [6]:
! cp ../data/claims.csv {base_preproc_input_dir}
! cp ../data/customers.csv {base_preproc_input_dir}

#### 로컬에서 스크립트 실행
- 전처리 코드에서 제공하는 로그를 통해서, 전처리 수행 내역을 확인 합니다.

In [7]:
! python {preprocessing_code} --base_preproc_input_dir {base_preproc_input_dir} \
                              --base_output_dir {base_output_dir} \
                              --split_rate {split_rate}


######### Argument Info ####################################
args.base_output_dir: opt/ml/processing/output
args.base_preproc_input_dir: opt/ml/processing/input
args.label_column: fraud
args.split_rate: 0.2

### Loading Claim Dataset
input_files: 
 ['opt/ml/processing/input/claims.csv']
dataframe shape 
 (5000, 17)
dataset sample 
           driver_relationship incident_type  ... incident_hour fraud
policy_id                                    ...                    
1                      Spouse     Collision  ...             8     0
2                        Self     Collision  ...            11     0

[2 rows x 17 columns]

### Loading Customer Dataset
input_files: 
 ['opt/ml/processing/input/customers.csv']
dataframe shape 
 (5000, 12)
dataset sample 
            customer_age  months_as_customer  ...  customer_education  auto_year
policy_id                                    ...                               
1                    54                  94  ...           Associate      

#### 전처리된 데이터 확인
- 실제로 전처리 된 파일의 내역을 확인 합니다.
- 훈련 및 테스트 세트의 fraud 의 분포를 확인 합니다. (0: non-fruad, 1: fraud)

In [8]:
preprocessed_train_path = os.path.join(base_output_dir + '/train/train.csv')
preprocessed_test_path = os.path.join(base_output_dir + '/test/test.csv')

preprocessed_train_df = pd.read_csv(preprocessed_train_path)
preprocessed_test_df = pd.read_csv(preprocessed_test_path)

preprocessed_train_df
                                       

,fraud,vehicle_claim,total_claim_amount,customer_age,months_as_customer,num_claims_past_year,num_insurers_past_5_years,policy_deductable,policy_annual_premium,customer_zip,...,collision_type_missing,incident_severity_Major,incident_severity_Minor,incident_severity_Totaled,authorities_contacted_Ambulance,authorities_contacted_Fire,authorities_contacted_None,authorities_contacted_Police,police_report_available_No,police_report_available_Yes
0,0,8913.668763,80513.668763,54,94,0,1,750,3000,99207,...,0,0,1,0,0,0,1,0,1,0
1,0,19746.724395,26146.724395,41,165,0,1,750,2950,95632,...,0,0,0,1,0,0,0,1,0,1
2,0,11652.969918,22052.969918,57,155,0,1,750,3000,93203,...,0,0,1,0,0,0,0,1,0,1
3,0,11260.930936,115960.930936,39,80,0,1,750,3000,85208,...,0,0,1,0,0,0,1,0,1,0
4,0,27987.704652,31387.704652,39,60,0,1,750,3000,91792,...,0,1,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,0,18052.611626,67152.611626,42,103,1,1,750,3000,93654,...,0,0,1,0,0,0,1,0,1,0
3996,0,34949.202468,51749.202468,23,6,0,3,750,3000,94305,...,0,0,0,1,1,0,0,0,1,0
3997,0,4063.701410,9963.701410,44,35,0,2,750,2550,95476,...,0,0,1,0,0,0,0,1,0,1
3998,0,17390.520451,20490.520451,22,38,0,1,750,3000,90680,...,0,1,0,0,0,0,0,1,0,1


첫번때 데이터 세트는 훈련 데이터 세트, 두번째는 테스트 데이터 세트 입니다. 각각의 fraud의 비율을 확인하세요.

In [11]:
print("Train Data Set: ")
dp(preprocessed_train_df['fraud'].value_counts())

print("\nTest Data Set:")
dp(preprocessed_test_df['fraud'].value_counts())

Train Data Set: 


0    3869
1     131
Name: fraud, dtype: int64


Test Data Set:


0    967
1     33
Name: fraud, dtype: int64

### (2) SageMaker Model Building Pipeline 에서 전치러 스텝 개발 및 실행
---



### 데이터 S3 버킷으로의 업로드
- default 버킷으로 업로드 하기 위한 설정입니다.



In [37]:
bucket = sagemaker.Session().default_bucket()
default_bucket = bucket# replace with an existing bucket if needed
prefix = 'sagemaker/sm-webinar/sm-pipeline'

boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/claims.csv')).upload_file('opt/ml/processing/input/claims.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/customers.csv')).upload_file('opt/ml/processing/input/customers.csv')

In [18]:
input_data_uri = "s3://" + os.path.join(bucket, prefix, 'train/')

In [19]:
input_data_uri

's3://sagemaker-ap-northeast-2-806174985048/sagemaker/sm-webinar/sm-pipeline/train/'

### 모델 빌딩 파이프라인 변수 생성
- 아래와 같이 변수를 정의하여, 파이프라인 정의시에 인자로 제공할 수 있습니다.



In [20]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)

processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount",
    default_value=1
)
processing_instance_type = ParameterString(
    name="ProcessingInstanceType",
    default_value="ml.m5.xlarge"
)

input_data = ParameterString(
    name="InputData",
    default_value=input_data_uri,
)


### 전처리 스텝 프로세서 정의
- 전처리의 내장 SKLearnProcessor 를 통해서 sklearn_processor 오브젝트를 생성 합니다.

In [21]:
from sagemaker.sklearn.processing import SKLearnProcessor

framework_version = "0.23-1"

sklearn_processor = SKLearnProcessor(
    framework_version=framework_version,
    instance_type=processing_instance_type,
    instance_count=processing_instance_count,
    base_job_name="sklearn-fraud-process",
    role=role,
)
print("input_data: \n", input_data)

input_data: 
 s3://sagemaker-ap-northeast-2-806174985048/sagemaker/sm-webinar/sm-pipeline/train/


### 전처리 스텝 단계 정의
- 처리 단계에서는 아래와 같은 주요 인자가 있습니다.
    - 단계 이름
    - processor 기술: 위에서 생성한 processor 오브젝트를 제공
    - inputs: S3의 경로를 기술하고, 다커안에서의 다운로드 폴더(destination)을 기술 합니다.
    - outputs: 처리 결과가 저장될 다커안에서의 폴더 경로를 기술합니다.
        - 다커안의 결과 파일이 저장 후에 자동으로 S3로 업로딩을 합니다.
    - job_arguments: 사용자 정의의 인자를 기술 합니다.
    - code: 전처리 코드의 경로를 기술 합니다.
- 처리 단계의 상세한 사항은 여기를 보세요. --> [처리 단계, Processing Step](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/build-and-manage-steps.html#step-type-processing)

In [22]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep
    

step_process = ProcessingStep(
    name="FraudScratchProcess",
    processor=sklearn_processor,
    inputs=[
        ProcessingInput(source=input_data_uri,destination='/opt/ml/processing/input'),
         ],
    outputs=[ProcessingOutput(output_name="train",
                              source='/opt/ml/processing/output/train'),
             ProcessingOutput(output_name="test",
                              source='/opt/ml/processing/output/test')],
    job_arguments=["--split_rate", f"{split_rate}"],    
    code= preprocessing_code
)


### 파리마터, 단계, 조건을 조합하여 최종 파이프라인 정의
- 파이프라인의 정의를 생성 합니다.
    - name: 파이프라인의 이름
    - parameters: 위에서 기술한 parameters를 기술 합니다.
    - steps: 위에서 정의한 처리 스텝을 기술합니다.
        - [알림] 정의한 stpes 이 복수개이면 복수개를 기술합니다. 만약에 step 간에 의존성이 있으면, 명시적으로 기술하지 않아도 같이 실행 됩니다.



In [24]:
project_prefix = 'agemaker-pipeline-step-by-step'

In [25]:
from sagemaker.workflow.pipeline import Pipeline


pipeline_name = project_prefix
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_type, 
        processing_instance_count,
        input_data,
    ],
    steps=[step_process],
)

#### (선택) 파이프라인 정의 확인 

파이프라인을 정의하는 JSON을 생성하고 파이프라인 내에서 사용하는 파라미터와 단계별 속성들이 잘 정의되었는지 확인할 수 있습니다.

In [26]:
import json


definition = json.loads(pipeline.definition())
definition

{'Version': '2020-12-01',
 'Metadata': {},
 'Parameters': [{'Name': 'ProcessingInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.m5.xlarge'},
  {'Name': 'ProcessingInstanceCount', 'Type': 'Integer', 'DefaultValue': 1},
  {'Name': 'InputData',
   'Type': 'String',
   'DefaultValue': 's3://sagemaker-ap-northeast-2-806174985048/sagemaker/sm-webinar/sm-pipeline/train/'}],
 'PipelineExperimentConfig': {'ExperimentName': {'Get': 'Execution.PipelineName'},
  'TrialName': {'Get': 'Execution.PipelineExecutionId'}},
 'Steps': [{'Name': 'FraudScratchProcess',
   'Type': 'Processing',
   'Arguments': {'ProcessingResources': {'ClusterConfig': {'InstanceType': {'Get': 'Parameters.ProcessingInstanceType'},
      'InstanceCount': {'Get': 'Parameters.ProcessingInstanceCount'},
      'VolumeSizeInGB': 30}},
    'AppSpecification': {'ImageUri': '366743142698.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3',
     'ContainerArguments': ['--split_rate', '0.2'],
     'Cont

#### 파이프라인을 SageMaker에 제출하고 실행하기 

파이프라인 정의를 SageMaker Pipelines 서비스에 제출하여 파이프라인을 생성하거나 파이프라인이 이미 존재하면 파이프라인 정의를 업데이트합니다. 함께 전달되는 역할(role)을 이용하여 AWS에서 파이프라인을 생성하고 작업의 각 단계를 실행할 것입니다.   

In [27]:
pipeline.upsert(role_arn=role)

{'PipelineArn': 'arn:aws:sagemaker:ap-northeast-2:806174985048:pipeline/agemaker-pipeline-step-by-step',
 'ResponseMetadata': {'RequestId': '9f1e5092-2cbf-43a3-8077-92641a04d4be',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '9f1e5092-2cbf-43a3-8077-92641a04d4be',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '103',
   'date': 'Thu, 17 Mar 2022 06:57:02 GMT'},
  'RetryAttempts': 0}}

디폴트값을 이용하여 파이프라인을 샐행합니다. 

In [28]:
execution = pipeline.start()

#### 파이프라인 운영: 파이프라인 대기 및 실행상태 확인

워크플로우의 실행상황을 살펴봅니다. 

In [29]:
execution.describe()
execution.wait()

실행이 완료될 때까지 기다립니다.

실행된 단계들을 리스트업합니다. 파이프라인의 단계실행 서비스에 의해 시작되거나 완료된 단계를 보여줍니다.

In [30]:
execution.list_steps()

[{'StepName': 'FraudScratchProcess',
  'StartTime': datetime.datetime(2022, 3, 17, 6, 57, 5, 54000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2022, 3, 17, 7, 1, 7, 270000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 0,
  'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:ap-northeast-2:806174985048:processing-job/pipelines-9m9xeo9z4eha-fraudscratchprocess-6c0i67rzxe'}}}]

### [옵션] SageMaker Studio에서 확인하기
- 아래의 그림 처럼 SageMaker Studio에 로긴후에 따라하시면, SageMaker Studio 에서도 실행 내역을 확인할 수 있습니다.
- [알림] 이번 실습은 SageMaker Studio 와 병행하여 진행하는 것을 권장 드립니다. 하지만, SageMaker Studio 에서 다커 컨테이너를 실행하는(로컬 모드) 부분은 현재 기능이 없기에 SageMaker Notebook Instance를 사용해서 실습했습니다. 다커 컨테이너의 실행 부분을 생략한다면 SageMaker Studio에서 SageMaker Building Pipeline 의 작업을 권장 드립니다. 이유는 가시적으로 워크플로를 확인하고, GUI 에서 제공하는 여러 링크들이 작업을 하는데에 효과적이기 때문입니다.
- SageMaker Studio 개발자 가이드 --> [SageMaker Studio](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/studio.html)
![view_studio.png](img/view_studio.png)


#### 아티펙트 경로 추출
- 다음 노트북에서 사용할 훈련 및 테스트의 전처리 S3 경로를 저장 합니다.

In [31]:
def get_proc_artifact(execution, client, kind=0):
    '''
    kind: 0 --> train
    kind: 2 --> test
    '''
    response = execution.list_steps()
    proc_arn = response[0]['Metadata']['ProcessingJob']['Arn']
    proc_job_name = proc_arn.split('/')[-1]
    # print("proc_job_name: ", proc_job_name)
    
    response = client.describe_processing_job(ProcessingJobName = proc_job_name)
    train_preproc_artifact = response['ProcessingOutputConfig']['Outputs'][kind]['S3Output']['S3Uri']    
    
    return train_preproc_artifact

import boto3
client = boto3.client("sagemaker")

train_preproc_dir_artifact = get_proc_artifact(execution, client, kind=0 )
test_preproc_dir_artifact = get_proc_artifact(execution, client, kind=1 )

print("train_preproc_dir_artifact: ", train_preproc_dir_artifact)
print("test_preproc__dir_artifact: ", test_preproc_dir_artifact)

%store train_preproc_dir_artifact
%store test_preproc_dir_artifact

train_preproc_dir_artifact:  s3://sagemaker-ap-northeast-2-806174985048/sklearn-fraud-process-2022-03-17-06-56-57-393/output/train
test_preproc__dir_artifact:  s3://sagemaker-ap-northeast-2-806174985048/sklearn-fraud-process-2022-03-17-06-56-57-393/output/test
Stored 'train_preproc_dir_artifact' (str)
Stored 'test_preproc_dir_artifact' (str)


In [38]:
%store project_prefix
%store input_data_uri
%store default_bucket

Stored 'project_prefix' (str)
Stored 'input_data_uri' (str)
Stored 'default_bucket' (str)


#### 아티펙트의 S3 저장 여부 확인

In [32]:
! aws s3 ls {train_preproc_dir_artifact} --recursive

2022-03-17 07:01:01     682602 sklearn-fraud-process-2022-03-17-06-56-57-393/output/train/train.csv


In [33]:
! aws s3 ls {test_preproc_dir_artifact} --recursive

2022-03-17 07:01:02     171439 sklearn-fraud-process-2022-03-17-06-56-57-393/output/test/test.csv
